In [6]:
import pandas as pd
import sys


df = pd.read_csv("./dataset/covid19/france_new_cases.csv")

df_use = df[300:]
df_use = df_use[["New_cases"]]
df_use.index = range(len(df_use))

df_use

,New_cases
0,15452
1,1368
2,13186
3,2711
4,27817
...,...
691,14204
692,6766
693,64772
694,49188


In [2]:
from models.model import Informer
model = Informer(1, 1, 1, 24, 24, 30)
model

Informer(
  (enc_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(1, 512, kernel_size=(3,), stride=(1,), padding=(1,), padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TemporalEmbedding(
      (hour_embed): FixedEmbedding(
        (emb): Embedding(24, 512)
      )
      (weekday_embed): FixedEmbedding(
        (emb): Embedding(7, 512)
      )
      (day_embed): FixedEmbedding(
        (emb): Embedding(32, 512)
      )
      (month_embed): FixedEmbedding(
        (emb): Embedding(13, 512)
      )
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (dec_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(1, 512, kernel_size=(3,), stride=(1,), padding=(1,), padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TemporalEmbedding(
      (hour_embed): FixedEmbedding(
        (emb): Embedding(24, 512)
   

In [3]:
from torch.utils.data import Dataset as Dataset
from torch.utils.data import DataLoader as DataLoader
import numpy as np
import torch

class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, item):
        return self.data[item]

    def __len__(self):
        return len(self.data)

def nn_seq_us(batch_size, step):
    print('data processing...')
    dataset = df_use
    # split
    train = dataset[:int(len(dataset) * 0.6)]
    val = dataset[int(len(dataset) * 0.6):int(len(dataset) * 0.85)]
    test = dataset[int(len(dataset) * 0.85):len(dataset)]
    m, n = np.max(train[train.columns[0]]), np.min(train[train.columns[0]])

    def process(data, batch_size, shuffle):
        load = data[data.columns[0]]
        load = load.tolist()
        data = data.values.tolist()
        load = (load - n) / (m - n)
        seq = []
        for i in range(len(data) - step):
            train_seq = []
            train_label = []
            for j in range(i, i + step):
                x = [load[j]]
                train_seq.append(x)
            # for c in range(2, 8):
            #     train_seq.append(data[i + step][c])
            train_label.append(load[i + step])
            train_seq = torch.FloatTensor(train_seq)
            train_label = torch.FloatTensor(train_label).view(-1)
            seq.append((train_seq, train_label))

        # print(seq[-1])
        seq = MyDataset(seq)
        seq = DataLoader(dataset=seq, batch_size=batch_size, shuffle=shuffle, num_workers=0, drop_last=True)

        return seq

    data_train = process(train, batch_size, True)
    data_validate = process(val, batch_size, True)
    data_test = process(test, batch_size, False)

    return data_train, data_validate, data_test, m, n

In [4]:
from torch.optim.lr_scheduler import StepLR as StepLR
from tqdm import tqdm
from copy import deepcopy

import torch
import torch.nn as nn

def get_val_loss(model, data_validate, loss_function, device):
    model.eval()
    val_loss = []
    for (seq, label) in data_validate:
        seq = seq.to(device)
        label = label.to(device)
        y_pred = model(seq)
        loss = loss_function(y_pred, label)
        val_loss.append(loss.item())

    return sum(val_loss)/len(val_loss)# MSE


def informer_train(data_train, data_validate, path, seq_len, d_model, nhead, dropout, encoder_num_layers, decoder_num_layers, input_size=1, hidden_size=10, num_layers=2, output_size = 1, batch_size = 5, optimizer = 'adam', max_epochs = 20, lr = 0.01, device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")):
    weight_decay = 0.00005
    step_size = 5
    gamma = 0.1


    model = Informer(1, 1, 1, 24, 24, 30).to(device)

    loss_function = nn.MSELoss().to(device)
    if optimizer == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    else:
        optimizer = torch.optim.SGD(model.parameters(), lr=lr,)
    scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)
    # training
    min_epochs = 10
    best_model = None
    min_val_loss = 10

    loss = 10086
    min_train_loss = 10

    for epoch in range(max_epochs):
        train_loss = []
        for (seq, label) in data_train:
            optimizer.zero_grad()

            seq = seq.to(device)
            label = label.to(device)
            y_pred = model(seq)
            loss = loss_function(y_pred, label)
            train_loss.append(loss.item())
            loss.backward()
            optimizer.step()

        scheduler.step()

        # validation

        if epoch > min_epochs and loss < min_train_loss:
            min_train_loss = loss
            best_model = deepcopy(model)
            print(type(model))

        if epoch % 10 == 0:
            print('epoch {:03d} train_loss {:.8f}'.format(epoch, np.mean(train_loss)))


        model.train()

    state = {'models': best_model.state_dict()}
    torch.save(state, path)

In [5]:
from itertools import chain
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline

def get_mse(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.sum((y_true - y_pred) ** 2) / len(y_true)

def get_mape(y_true, y_pred):
    """
    Compute mean absolute percentage error (MAPE)
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
# def lstm_test(data_test, path, m, n, input_size = 1,  hidden_size = 10, num_layers = 2, output_size = 1, batch_size = 5, device = torch.device('cpu')):
def transformer_test(data_test, path, m, n, input_size, output_size, seq_len, d_model, nhead, dropout, encoder_num_layers, decoder_num_layers, device):
    pred = []
    y = []
    print('loading models...')

    model = Informer(1, 1, 1, 24, 24, 30).to(device)

    loss_function = nn.MSELoss().to(device)

    model.load_state_dict(torch.load(path)['models'])
    model.eval()
    print('predicting...')
    for (seq, target) in data_test:
        target = list(chain.from_iterable(target.data.tolist()))
        y.extend(target)
        seq = seq.to(device)
        with torch.no_grad():
            y_pred = model(seq)
            y_pred = list(chain.from_iterable(y_pred.data.tolist()))
            pred.extend(y_pred)

    y, pred = np.array(y), np.array(pred)
    y = (m - n) * y + n
    pred = (m - n) * pred + n

    # print('mape:', get_mape(y, pred))
    # print('mse:', get_mse(y, pred))

    # plot
    x = [i for i in range(1, y.shape[0] + 1)]
    x_smooth = np.linspace(np.min(x), np.max(x), 900)
    y_smooth = make_interp_spline(x, y)(x_smooth)
    plt.plot(x_smooth, y_smooth, c='green', marker='*', ms=1, alpha=0.75, label='true')

    y_smooth = make_interp_spline(x, pred)(x_smooth)
    plt.plot(x_smooth, y_smooth, c='red', marker='o', ms=1, alpha=0.75, label='pred')
    plt.grid(axis='y')
    plt.legend()
    # plt.savefig("./new.png")
    plt.show()



def future_transformer_test(data_test, path, m, n, input_size, output_size, seq_len, d_model, nhead, dropout, encoder_num_layers, decoder_num_layers, device):
    pred = []
    y = []
    print('loading models...')

    model = Informer(1, 1, 1, 24, 24, 30).to(device)

    model.load_state_dict(torch.load(path)['models'])
    model.eval()
    print('predicting...')

    y_pred = 0
    seq = []
    label = []
    ss = [(seq, target) for (seq, target) in data_test]
    for i in range(len(ss)):
        label.append(ss[i][1][0])

    y = label

    # label = list(chain.from_iterable(label.tolist()))
    ss = ss[0]
    for i in range(50):

        seq = ss[0].to(device)
        with torch.no_grad():
            y_pred = model(seq)
            # print(f'seq{seq};')
            for i in range(len(seq[0]) - 1):
                seq[0][i] = seq[0][i+1]
            seq[0][-1] = y_pred
            # print(f'pred: {y_pred}')
            y_pred = list(chain.from_iterable(y_pred.data.tolist()))
            pred.extend(y_pred)

    # print(seq)
    # for i in range(len(seq[0]) - 1):
    #     seq[0][i] = seq[0][i+1]
    # seq[0][-1] = y_pred[0]


    # print(f'y_pred is {y_pred}')

    y = []
    for i in range(len(label)):
        y.append(label[i].detach().numpy()[0])

    print(len(y))
    print(len(pred))


    y, pred = np.array(y), np.array(pred)
    y = (m - n) * y + n
    pred = (m - n) * pred + n

    # print('mape:', get_mape(y, pred))
    # print('mse:', get_mse(y, pred))


    # plot
    # x = [i for i in range(1, y.shape[0] + 1)]
    x = [i for i in range(1, 50+1)]
    x_smooth = np.linspace(np.min(x), np.max(x), 900)
    y_smooth = make_interp_spline(x, y[:50])(x_smooth)
    plt.plot(x_smooth, y_smooth, c='green', marker='*', ms=1, alpha=0.75, label='true')

    # x = [i for i in range(1, pred.shape[0] + 1)]
    x = [i for i in range(1, 50+1)]
    y_smooth = make_interp_spline(x, pred[:50])(x_smooth)
    plt.plot(x_smooth, y_smooth, c='red', marker='o', ms=1, alpha=0.75, label='pred')
    plt.grid(axis='y')
    plt.legend()
    # plt.savefig("./new.png")
    plt.show()



In [6]:
input_size=1
hidden_size=10
num_layers=2
output_size = 1
batch_size = 1
optimizer = 'adam'
max_epochs = 25
lr = 0.001
# lr = 0.03

step = 32    # 每step_size个epoch调整一次学习率

seq_len = 32  #
d_model = 16

nhead = 2
dropout = 0.2

encoder_num_layers = 2
decoder_num_layers = 1


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

model_path = './informer_model/best_model_transformer.model'

data_train, data_validate, data_test, m, n = nn_seq_us(batch_size=batch_size, step=step)

informer_train(data_train, data_validate, model_path, seq_len, d_model, nhead, dropout, encoder_num_layers, decoder_num_layers, input_size, hidden_size, num_layers, output_size, batch_size, optimizer, max_epochs, lr, device)

cpu
data processing...


TypeError: forward() missing 3 required positional arguments: 'x_mark_enc', 'x_dec', and 'x_mark_dec'